## **Ignore Warnings**

In [7]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Ignore all UserWarnings
warnings.filterwarnings("ignore", message=".*DeprecationWarning.*") # Ignore warnings containing "DeprecationWarning"

## **Installing required libraries**

In [8]:
!pip install nltk
!pip install sumy
!pip install rouge

## **Importing Required Libraries**

In [9]:
import numpy as np
import pandas as pd
import os
import nltk
import re
import zipfile
# from google.colab import drive             #UNCOMMENT IF USING DRIVE
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

## **Downloading required NLTK components**

In [10]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/itadmin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/itadmin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/itadmin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## **Connect to drive**

Run this cell only if you are running it on Colab and ensure the zip file of validation dataset is uploaded on colab

In [12]:
use_colab=False

In [11]:
# drive.mount('/content/drive')
# # Path to the directory containing the dataset on Google Drive
# dataset_path = '/content/drive/MyDrive/NLP project/validation.zip' #path to your dataset
# use_colab=True

## **Unzip to extract files**

In [13]:
if(use_colab):
    # Unzip the uploaded file
    zip_file_name = "/content/drive/MyDrive/NLP project/validation.zip" # replace with the folder path on drive
    output_folder = '/content/unzipped_files/'

    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(output_folder)

## **Loading Annual Reports in Data Frame**

In [14]:
# Specify the path to the folder containing the text files
if(use_colab): folder_path = r'/content/unzipped_files/validation/annual_reports'
else: folder_path = r'validation/annual_reports'  # Replace with the actual path to your folder

# Create an empty list to store the data
data = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Check if it's a text file
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding = 'utf-8') as f:
            text = f.read()
        data.append({'filename': filename, 'text': text})

# Create a dataframe from the list of dictionaries
df = pd.DataFrame(data)


In [15]:
df

,filename,text
0,32561.txt,Interactive PDF\nUser guide\nThis PDF allows ...
1,30858.txt,Annual Report and Accounts 2017\nEmpowering \...
2,32148.txt,Connecting \neverybody to \nlive a better \n...
3,32657.txt,Unquenchable \nthirst for \nimprovement\nA.G....
4,32773.txt,Circle Property Plc Annual Report and Account...
...,...,...
358,30820.txt,Delivering \nquality\nAnnual Report and Acco...
359,30927.txt,Ibstock plc Annual Report and Accounts 2017\n...
360,32176.txt,Annual Report 2017\nTate & Lyle Annual Report...
361,31654.txt,easyHotel plc\nAccelerated \ngrowth \nAnnual...


## **Preprocessing**

In [16]:
def remove_spaces_links(df):
    df["text"] = df["text"].str.replace("\n", " ")
    df["text"] = df["text"].str.replace(r"\s+", " ")
    df.loc[:,'text'] = df['text'].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    return df

# def replace_special_symbols(df):
#     df["text"] = df["text"].str.replace("@", "")
#     df["text"] = df["text"].str.replace("#", "")
#     df["text"] = df["text"].str.replace("$", "")
#     df["text"] = df["text"].str.replace("~", "")
#     return df

def lower_case_text(df):
    df['text'] = df['text'].str.lower()
    return df

# def lemmatize_text(df):
#     lemmatizer = WordNetLemmatizer()
#     df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
#     return df


In [17]:
df = remove_spaces_links(df)
# df = replace_special_symbols(df)
df=lower_case_text(df)
# df =lemmatize_text(df)

/tmp/ipykernel_5374/2316140606.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r"\s+", " ")
/tmp/ipykernel_5374/2316140606.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,'text'] = df['text'].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()


In [18]:
df

,filename,text
0,32561.txt,interactive pdf user guide this pdf allows you...
1,30858.txt,annual report and accounts 2017 empowering the...
2,32148.txt,connecting everybody to live a better today an...
3,32657.txt,unquenchable thirst for improvement a.g. barr ...
4,32773.txt,circle property plc annual report and accounts...
...,...,...
358,30820.txt,delivering quality annual report and accounts ...
359,30927.txt,ibstock plc annual report and accounts 2017 de...
360,32176.txt,annual report 2017 tate & lyle annual report 2...
361,31654.txt,easyhotel plc accelerated growth annual report...


## **Generate and save the summaries**

### **LSA summarizer**

In [19]:
def summarize_with_lsa(text, sentences_count=70):
    # Parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    # Count the number of sentences in the text
    sentence_count = len(list(parser.document.sentences))

    # Calculate 7% of the sentence count
    sentences_count = int(0.07 * sentence_count)

    # Create an LSA Summarizer
    summarizer = LsaSummarizer()

    # Get the summary
    summary = summarizer(parser.document, sentences_count)

    # Convert the summary to a list of sentences
    summary_sentences = [str(sentence) for sentence in summary]

    # Join the sentences to form the final summary
    final_summary = ' '.join(summary_sentences)

    return final_summary

In [20]:
def summarize_and_save(row, text_column="text", filename_column="filename"):
    """Summarizes the text from a dataframe row and saves it with the specified filename."""

    summary = summarize_with_lsa(row[text_column])  # Summarize using LSA
    filename = row[filename_column]
    summary_filename = f"{filename[:-4]}_summary.txt"

    os.makedirs("gen_sum", exist_ok=True)# will create a gen_sum folder in your current directory of project
    with open(f"gen_sum/{summary_filename}", "w",encoding='utf-8') as f:
        f.write(summary)

# Example usage with dataframe:
for i, row in df.iterrows():
    summarize_and_save(row)  # Pass the entire row to the function

## **Load generated Summaries**

In [21]:
folder_path = 'gen_sum'
data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r',encoding="utf-8") as f:
            text = f.read()
        data.append({'filename': filename, 'summary': text})

gen_summaries = pd.DataFrame(data)

In [22]:
gen_summaries

,filename,summary
0,31474_summary.txt,we apply world-class materials science and man...
1,32340_summary.txt,bring energy to life annual report and account...
2,32333_summary.txt,we have also strengthened the board with the a...
3,31617_summary.txt,"certain figures contained in this document, in..."
4,30943_summary.txt,"since then, it has consistently maintained its..."
...,...,...
358,32480_summary.txt,integrated reporting has become modern best pr...
359,31005_summary.txt,"further explanations can be found in notes 4, ..."
360,32345_summary.txt,"babcock provides skilled, bespoke engineering ..."
361,30785_summary.txt,→ housing operating margins reduced to 5.2% (2...


## **Load gold summaries**

In [23]:
if(use_colab): folder_path = r'/content/unzipped_files/validation/gold_summaries'
else: folder_path = r'validation/gold_summaries'  # Replace with the actual path to your folder

data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r',encoding="utf-8") as f:
            text = f.read()
        data.append({'filename': filename, 'gold_summary': text})

gold_summaries = pd.DataFrame(data)

In [24]:
gold_summaries

,filename,gold_summary
0,31277_2.txt,2 Velocys plc Annual report and accounts 2017...
1,32939_1.txt,16\nmanx telecom plc\nannual report and accou...
2,30887_1.txt,“BAE Systems delivered a \ngood performance ...
3,32097_2.txt,25361.02 13-6-17 Proof Four\nUNDERLYING...
4,31005_2.txt,Building sustainable growth\nCommercial highl...
...,...,...
1245,32025_3.txt,CHAIRMAN’S STATEMENT\nOverview\nIn my last ye...
1246,31064_3.txt,Our Business \nThe Group has performed well a...
1247,32236_1.txt,SysGroup Plc Annual Report & Accounts 2017 9...
1248,32848_2.txt,* Underlying results are stated before \nacqu...


## **Sort by filename**

In [25]:
df_sorted = gold_summaries.sort_values(by='filename')

In [26]:
df_sorted

,filename,gold_summary
188,30777_1.txt,25695 19 March 2018 3:29 PM Proof 7\n02...
741,30777_2.txt,25695 19 March 2018 3:29 PM Proof 7\nFi...
886,30777_3.txt,25695 19 March 2018 3:29 PM Proof 7\nA....
509,30778_1.txt,10\nStaffline Group plc Annual Report 2017\nC...
290,30778_2.txt,3 Overview Strategic Report Corporate Governa...
...,...,...
1167,33154_3.txt,Dixons Carphone plc Annual Report and Accoun...
463,33155_1.txt,"IN CONVERSATION WITH \nDAVID ATKINS, CHIEF EX..."
226,33155_2.txt,HIGHLIGHTS\n2015 overview\nPortfolio value\n(...
560,33155_3.txt,Setting Hammerson’s \nvalues and standards\nC...


In [27]:
gen_summaries

,filename,summary
0,31474_summary.txt,we apply world-class materials science and man...
1,32340_summary.txt,bring energy to life annual report and account...
2,32333_summary.txt,we have also strengthened the board with the a...
3,31617_summary.txt,"certain figures contained in this document, in..."
4,30943_summary.txt,"since then, it has consistently maintained its..."
...,...,...
358,32480_summary.txt,integrated reporting has become modern best pr...
359,31005_summary.txt,"further explanations can be found in notes 4, ..."
360,32345_summary.txt,"babcock provides skilled, bespoke engineering ..."
361,30785_summary.txt,→ housing operating margins reduced to 5.2% (2...


## **Merge to create single frame**

In [28]:
gen_summaries['base_filename'] = gen_summaries['filename'].str.split('_').str[0]
df_sorted['base_filename'] = df_sorted['filename'].str.split('_').str[0]
merged_df = pd.merge(gen_summaries, df_sorted, on='base_filename', how='inner')
result_df = merged_df[['filename_x', 'filename_y', 'summary', 'gold_summary']]

In [29]:
result_df

,filename_x,filename_y,summary,gold_summary
0,31474_summary.txt,31474_1.txt,we apply world-class materials science and man...,We continue to make good progress with the \n...
1,31474_summary.txt,31474_2.txt,we apply world-class materials science and man...,"1. Throughout the Annual Report, including th..."
2,31474_summary.txt,31474_3.txt,we apply world-class materials science and man...,Financial performance showed progress with \n...
3,32340_summary.txt,32340_1.txt,bring energy to life annual report and account...,Chief Executive’s review\nThe past year was a...
4,32340_summary.txt,32340_2.txt,bring energy to life annual report and account...,1. UK Electricity Transmission 29\n2. UK Gas ...
...,...,...,...,...
1245,30785_summary.txt,30785_3.txt,→ housing operating margins reduced to 5.2% (2...,Chairman’s statement\n“ We continue to take a...
1246,30785_summary.txt,30785_5.txt,→ housing operating margins reduced to 5.2% (2...,"Q&A with CEO, David Miles\n92%\nof tenants ra..."
1247,32556_summary.txt,32556_1.txt,overview 02 investment case 02 group at a glan...,Strategic Report\nwww.biffa.co.uk\n10\nBiffa ...
1248,32556_summary.txt,32556_2.txt,overview 02 investment case 02 group at a glan...,FY17 Net Revenue\n1\n (%)\n£898.8m\nFY17 Unde...


## **Evaluating using Rouge Score**

In [30]:
from rouge import Rouge
import sys

# Increase recursion limit
sys.setrecursionlimit(10**6)
rouge = Rouge()

def calculate_rouge(row):
    scores = rouge.get_scores(row['summary'], row['gold_summary'])[0]
    return scores

result_df['rouge_scores'] = result_df.apply(calculate_rouge, axis=1)

print(result_df['rouge_scores'])

0       {'rouge-1': {'r': 0.5548780487804879, 'p': 0.1...
1       {'rouge-1': {'r': 0.632183908045977, 'p': 0.04...
2       {'rouge-1': {'r': 0.5341880341880342, 'p': 0.0...
3       {'rouge-1': {'r': 0.5574912891986062, 'p': 0.1...
4       {'rouge-1': {'r': 0.4987146529562982, 'p': 0.0...
                              ...                        
1245    {'rouge-1': {'r': 0.5185589519650655, 'p': 0.2...
1246    {'rouge-1': {'r': 0.6210268948655256, 'p': 0.1...
1247    {'rouge-1': {'r': 0.5213270142180095, 'p': 0.1...
1248    {'rouge-1': {'r': 0.4654255319148936, 'p': 0.1...
1249    {'rouge-1': {'r': 0.4990512333965844, 'p': 0.1...
Name: rouge_scores, Length: 1250, dtype: object


In [31]:
rouge_scores = result_df['rouge_scores']
rouge_scores

0       {'rouge-1': {'r': 0.5548780487804879, 'p': 0.1...
1       {'rouge-1': {'r': 0.632183908045977, 'p': 0.04...
2       {'rouge-1': {'r': 0.5341880341880342, 'p': 0.0...
3       {'rouge-1': {'r': 0.5574912891986062, 'p': 0.1...
4       {'rouge-1': {'r': 0.4987146529562982, 'p': 0.0...
                              ...                        
1245    {'rouge-1': {'r': 0.5185589519650655, 'p': 0.2...
1246    {'rouge-1': {'r': 0.6210268948655256, 'p': 0.1...
1247    {'rouge-1': {'r': 0.5213270142180095, 'p': 0.1...
1248    {'rouge-1': {'r': 0.4654255319148936, 'p': 0.1...
1249    {'rouge-1': {'r': 0.4990512333965844, 'p': 0.1...
Name: rouge_scores, Length: 1250, dtype: object

## **Calculate Average Rouge Score**

In [44]:
# Initialize accumulated scores
total_rouge_1 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_2 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_l = {'r': 0, 'p': 0, 'f': 0}

# Accumulate scores
for scores_dict in rouge_scores:
    total_rouge_1['r'] += scores_dict['rouge-1']['r']
    total_rouge_1['p'] += scores_dict['rouge-1']['p']
    total_rouge_1['f'] += scores_dict['rouge-1']['f']
    
    total_rouge_2['r'] += scores_dict['rouge-2']['r']
    total_rouge_2['p'] += scores_dict['rouge-2']['p']
    total_rouge_2['f'] += scores_dict['rouge-2']['f']
    
    total_rouge_l['r'] += scores_dict['rouge-l']['r']
    total_rouge_l['p'] += scores_dict['rouge-l']['p']
    total_rouge_l['f'] += scores_dict['rouge-l']['f']

# Calculate average scores
num_scores = len(rouge_scores)

avg_rouge_1 = {key: total_rouge_1[key] / num_scores for key in total_rouge_1}
avg_rouge_2 = {key: total_rouge_2[key] / num_scores for key in total_rouge_2}
avg_rouge_l = {key: total_rouge_l[key] / num_scores for key in total_rouge_l}

print("Average ROUGE-1 scores:", avg_rouge_1)
print("Average ROUGE-2 scores:", avg_rouge_2)
print("Average ROUGE-L scores:", avg_rouge_l)

Average ROUGE-1 scores: {'r': 0.4963936352727142, 'p': 0.144899851207217, 'f': 0.20711740215116875}
Average ROUGE-2 scores: {'r': 0.2147374252541646, 'p': 0.05124208201023698, 'f': 0.07327136043495615}
Average ROUGE-L scores: {'r': 0.46540715023313367, 'p': 0.13494999568778007, 'f': 0.19303599726417722}


## **Calculate average for csv file generated via Java tool**

In [45]:
import pandas as pd

# Assuming 'your_file.csv' is the name of your CSV file
df = pd.read_csv('results.csv')

# Filter data based on ROUGE types
rouge_1_data = df[df['ROUGE-Type'].str.contains('ROUGE-1')]
rouge_2_data = df[df['ROUGE-Type'].str.contains('ROUGE-2')]
rouge_l_data = df[df['ROUGE-Type'].str.contains('ROUGE-L')]
rouge_su_data = df[df['ROUGE-Type'].str.contains('ROUGE-SU')]

print(type(rouge_1_data))

print(rouge_l_data)

# Calculate averages for each ROUGE type
# print("op",rouge_l_data['Avg_F-Score'])
avg_rouge_1 = rouge_1_data['Avg_F-Score'].mean()
avg_rouge_2 = rouge_2_data['Avg_F-Score'].mean()
avg_rouge_l = rouge_l_data['Avg_F-Score'].mean()
avg_rouge_su = rouge_su_data['Avg_F-Score'].mean()

print(type(avg_rouge_1))

print(avg_rouge_1)

# Print the results
print(f'Average ROUGE-1 F-Score: {avg_rouge_1:.5f}')
print(f'Average ROUGE-2 F-Score: {avg_rouge_2:.5f}')
print(f'Average ROUGE-L F-Score: {avg_rouge_l:.5f}')
print(f'Average ROUGE-SU F-Score: {avg_rouge_su:.5f}')


<class 'pandas.core.frame.DataFrame'>
                            ROUGE-Type  Task Name  System Name  Avg_Recall  \
0     ROUGE-L+StopWordRemoval+Stemming      33090  SUMMARY.TXT     0.76192   
4     ROUGE-L+StopWordRemoval+Stemming      30817  SUMMARY.TXT     0.52661   
8     ROUGE-L+StopWordRemoval+Stemming      30816  SUMMARY.TXT     0.67083   
12    ROUGE-L+StopWordRemoval+Stemming      30819  SUMMARY.TXT     0.61295   
16    ROUGE-L+StopWordRemoval+Stemming      33097  SUMMARY.TXT     0.43248   
20    ROUGE-L+StopWordRemoval+Stemming      33098  SUMMARY.TXT     0.64501   
24    ROUGE-L+StopWordRemoval+Stemming      32688  SUMMARY.TXT     0.61376   
28    ROUGE-L+StopWordRemoval+Stemming      31478  SUMMARY.TXT     0.48476   
32    ROUGE-L+StopWordRemoval+Stemming      31114  SUMMARY.TXT     0.56095   
36    ROUGE-L+StopWordRemoval+Stemming      32562  SUMMARY.TXT     0.60987   
40    ROUGE-L+StopWordRemoval+Stemming      32684  SUMMARY.TXT     0.58318   
44    ROUGE-L+StopWordRemo